In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps to use your Google API key. You only have to do this once
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV saved/created in part two into a DataFrame.
vacation_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
vacation_data_df.head()

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
0,4,4,Pochutla,MX,15.74,-96.47,87.01,62,40,11.41,scattered clouds,0.0,0,Hotel Posada San Jose
1,5,5,Sakakah,SA,29.97,40.21,91.40,18,0,10.29,clear sky,0.0,0,Fakhamat Al Orjoana Apartment
2,7,7,Kahului,US,20.89,-156.47,89.60,40,20,24.16,few clouds,0.0,0,Maui Seaside Hotel
3,8,8,Arraial Do Cabo,BR,-22.97,-42.03,75.20,69,0,13.87,clear sky,0.0,0,Pousada Porto Praia
4,11,11,Vaini,TO,-21.20,-175.20,73.40,78,75,14.99,broken clouds,0.0,0,Keleti Beach Resort


In [4]:
# Create separte dataframe for first city
# city1_data_df = vacation_data_df[vacation_data_df.City_ID == 23]
city1_data_df = vacation_data_df.loc[(vacation_data_df["City"] == 'Jamestown')]
city1_data_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
23,66,66,Jamestown,US,42.1,-79.24,70.0,93,90,14.99,light rain,0.0,0,Hampton Inn & Suites Jamestown


In [5]:
# Create separte dataframe for second city
# city2_data_df = vacation_data_df[vacation_data_df.City == 'Corning']
city2_data_df = vacation_data_df.loc[(vacation_data_df["City"] == 'Corning')]
city2_data_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
110,282,282,Corning,US,42.14,-77.05,78.8,57,90,24.16,overcast clouds,0.0,0,Fairfield Inn by Marriott Corning Riverside


In [6]:
# Create separte dataframe for 3rd city
# city3_data_df = vacation_data_df[vacation_data_df.City == 'Broome']
city3_data_df = vacation_data_df.loc[(vacation_data_df["City"] == 'Broome')]
city3_data_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
85,221,221,Broome,US,42.25,-75.83,78.01,73,75,10.29,broken clouds,0.0,0,Chenango Valley State Park


In [7]:
# Create separte dataframe for fourth city
# city4_data_df = vacation_data_df [vacation_data_df.City == 'Albany']
city4_data_df = vacation_data_df.loc[(vacation_data_df["City"] == 'Bethel')]
city4_data_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
27,77,77,Bethel,US,41.37,-73.41,82.0,74,20,9.17,few clouds,0.0,0,Courtyard by Marriott Danbury


In [8]:
# Get the latitude-longitude pairs for each city DataFrame
city1_pair = city1_data_df["Lat"].values[0],city1_data_df["Lng"].values[0]
city2_pair = city2_data_df["Lat"].values[0],city2_data_df["Lng"].values[0]             
city3_pair = city3_data_df["Lat"].values[0],city3_data_df["Lng"].values[0] 
city4_pair = city4_data_df["Lat"].values[0],city4_data_df["Lng"].values[0] 
city1_pair

(42.1, -79.24)

In [9]:
# Create itinerary map of four cities
fig = gmaps.figure()
city1_to_city4 = gmaps.directions_layer(city1_pair, city4_pair, waypoints=[city2_pair, city3_pair],
        travel_mode='DRIVING')
fig.add_layer(city1_to_city4)
fig


Figure(layout=FigureLayout(height='420px'))

In [15]:
# concatenate city dataframes for pop-up marker
frames = [city1_data_df, city2_data_df, city3_data_df, city4_data_df]


combined_cities_df = pd.concat(frames)
combined_cities_df

,City_ID,City_ID.1,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
23,66,66,Jamestown,US,42.10,-79.24,70.00,93,90,14.99,light rain,0.0,0,Hampton Inn & Suites Jamestown
110,282,282,Corning,US,42.14,-77.05,78.80,57,90,24.16,overcast clouds,0.0,0,Fairfield Inn by Marriott Corning Riverside
85,221,221,Broome,US,42.25,-75.83,78.01,73,75,10.29,broken clouds,0.0,0,Chenango Valley State Park
27,77,77,Bethel,US,41.37,-73.41,82.00,74,20,9.17,few clouds,0.0,0,Courtyard by Marriott Danbury


In [16]:
# add a pop-up marker for each city that displays the hotel name, city name, country,
# current weather description and maximum temperature.
# Define template for pop-up box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row. use the iterrows() function to get the index and data in the row to add to the marker.
# hotel_info is a list
hotel_info = [info_box_template.format(**row) for index, row in combined_cities_df.iterrows()]
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Hampton Inn & Suites Jamestown</dd>\n<dt>City</dt><dd>Jamestown</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>light rain and 70.0 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Fairfield Inn by Marriott Corning Riverside</dd>\n<dt>City</dt><dd>Corning</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 78.8 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Chenango Valley State Park</dd>\n<dt>City</dt><dd>Broome</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>broken clouds and 78.01 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Courtyard by Marriott Danbury</dd>\n<dt>City</dt><dd>Bethel</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>few clouds and 82.0 °F</dd>\n</dl>\n']

In [17]:
locations = combined_cities_df[["Lat","Lng"]]
locations

,Lat,Lng
23,42.10,-79.24
110,42.14,-77.05
85,42.25,-75.83
27,41.37,-73.41


In [21]:
# Add a map with directions between four cities and a pop-up marker for each city.

# Get the latitude and longitude.
locations = combined_cities_df[["Lat", "Lng"]]

fig = gmaps.figure()

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))